https://blog.tensorflow.org/2020/09/introducing-tensorflow-recommenders.html

In [ ]:
!unzip '/content/drive/MyDrive/GdriveXbot/Indus_OS_Participants_Data.zip'

Archive:  /content/drive/MyDrive/GdriveXbot/Indus_OS_Participants_Data.zip
   creating: Indus_OS_Participants_Data/
  inflating: __MACOSX/._Indus_OS_Participants_Data  
  inflating: Indus_OS_Participants_Data/IndusOS_Data Dictionary.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._IndusOS_Data Dictionary.csv  
  inflating: Indus_OS_Participants_Data/app_installs.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._app_installs.csv  
  inflating: Indus_OS_Participants_Data/.DS_Store  
  inflating: __MACOSX/Indus_OS_Participants_Data/._.DS_Store  
  inflating: Indus_OS_Participants_Data/validation_data.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._validation_data.csv  
  inflating: Indus_OS_Participants_Data/actual_set.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._actual_set.csv  
  inflating: Indus_OS_Participants_Data/app_metadata.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._app_metadata.csv  
  inflating: Indus_OS_Participants_Data/app_usa

In [ ]:
!pip install gurobipy seaborn lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 18.2 MB/s 
     |████████████████████████████████| 310 kB 51.8 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705377 sha256=3929db066c07de6225bc19bde0846a093ec416a9c3af6e5713f3e7bc040e60c4
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [ ]:
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import auc_score  

In [ ]:
#
import numpy as np 
import pandas as pd 
import warnings 
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt 
import seaborn as sns

In [ ]:
dic=pd.read_csv('/content/Indus_OS_Participants_Data/IndusOS_Data Dictionary.csv')
app_ins=pd.read_csv('/content/Indus_OS_Participants_Data/app_installs.csv')
app_data=pd.read_csv('/content/Indus_OS_Participants_Data/app_metadata.csv')
app_u=pd.read_csv('/content/Indus_OS_Participants_Data/app_usage.csv')
ss=pd.read_csv('/content/Indus_OS_Participants_Data/sample_submission.csv')
u_data=pd.read_csv('/content/Indus_OS_Participants_Data/user_metadata.csv')
val=pd.read_csv('/content/Indus_OS_Participants_Data/validation_data.csv')
aa=pd.read_csv('/content/Indus_OS_Participants_Data/actual_set.csv')

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(1, 3))
#app_u['s_time_spent'] = scaler.fit_transform(app_u['time_spent'].values.reshape(-1,1))

In [ ]:
app_u = app_u.loc[app_u['time_spent'] > 10000] 

#Converting milliseconds to minutes
app_u['time_spent'] = app_u['time_spent'].div(60000).round(2)


In [ ]:
app_ins['time_spent'] = np.where(app_ins.status=='uninstalled',-0.5,0.5)

In [ ]:
app_ins_n = app_ins[['uid','item_id','time_spent']]
app_u = app_u[['uid','item_id','time_spent']]

In [ ]:
first = pd.concat([app_u,app_ins_n],axis=0)

In [ ]:
df=first.groupby(['uid','item_id'])['time_spent'].agg(['sum'])

In [ ]:
df=df.reset_index(drop=False)
df.head()

,uid,item_id,sum
0,2,5526,124.29
1,2,20875,6.44
2,2,25349,6.91
3,2,29763,0.36
4,2,145082,4.03


In [ ]:
app_count = (df.
     groupby(by = ['item_id'])['uid'].
     count().
     reset_index().
     rename(columns = {'uid': 'app_count'})
     [['item_id', 'app_count']]
    )
app_count.head()

,item_id,app_count
0,142,2
1,234,5
2,294,1
3,312,1
4,332,1


In [ ]:
rating_with_totalRatingCount = df.merge(app_count, left_on = 'item_id', right_on = 'item_id', how = 'left')
rating_with_totalRatingCount.head()

,uid,item_id,sum,app_count
0,2,5526,124.29,107788
1,2,20875,6.44,219907
2,2,25349,6.91,125356
3,2,29763,0.36,57346
4,2,145082,4.03,81858


In [ ]:
popularity_threshold = 30
rating_popular_movie= rating_with_totalRatingCount.query('app_count >= @popularity_threshold')
rating_popular_movie.head()

,uid,item_id,sum,app_count
0,2,5526,124.29,107788
1,2,20875,6.44,219907
2,2,25349,6.91,125356
3,2,29763,0.36,57346
4,2,145082,4.03,81858


In [ ]:
user_count=(df.
     groupby(by = ['uid'])['item_id'].
     count().
     reset_index().
     rename(columns = {'item_id': 'total_user_Count'})
     [['uid', 'total_user_Count']]
    )
user_count.head()

,uid,total_user_Count
0,2,11
1,3,30
2,4,20
3,6,19
4,7,53


In [ ]:
df = rating_popular_movie.merge(user_count, left_on = 'uid', right_on = 'uid', how = 'left')
df.head()

,uid,item_id,sum,app_count,total_user_Count
0,2,5526,124.29,107788,11
1,2,20875,6.44,219907,11
2,2,25349,6.91,125356,11
3,2,29763,0.36,57346,11
4,2,145082,4.03,81858,11


In [ ]:
user_threshold = 2
df= df.query('total_user_Count >= @user_threshold')
df.head()

,uid,item_id,sum,app_count,total_user_Count
0,2,5526,124.29,107788,11
1,2,20875,6.44,219907,11
2,2,25349,6.91,125356,11
3,2,29763,0.36,57346,11
4,2,145082,4.03,81858,11


In [ ]:
df.total_user_Count.m()

1024

In [ ]:
adr = df[df.total_user_Count>70]

In [ ]:
eer = adr.uid.unique()
g = pd.DataFrame()
g['uid'] = eer

In [ ]:
g

In [ ]:
eer

In [ ]:
val.head()

,uid
0,255352
1,290183
2,210428
3,17401
4,48078


In [ ]:
v_new = pd.concat([val,g],axis=0)

In [ ]:
df_n=df[df.uid.isin(v_new['uid']) == True]

In [ ]:
print('Number of unique books: ', df['item_id'].nunique())
print('Number of unique users: ', df['uid'].nunique())

Number of unique books:  9309
Number of unique users:  236636


In [ ]:
print('Number of unique books: ', df_n['item_id'].nunique())
print('Number of unique users: ', df_n['uid'].nunique())

Number of unique books:  9309
Number of unique users:  49593


In [ ]:
app_u.uid.nunique()

238640

In [ ]:
aa.uid.nunique()

169542

In [ ]:
val.uid.nunique()

42252

In [ ]:
movie_features_df=df_n.pivot_table(index='uid',columns='item_id',values='sum').fillna(0)
movie_features_df.head()

item_id,701,1048,5115,5526,5536,5918,6901,6941,6955,6958,...,930534,930550,930554,930569,930606,930621,930624,930625,930915,930951
uid,,,,,,,,,,,,,,,,,,,,,
11,0.0,0.0,0.0,167.50,0.0,0.0,0.0,0.0,0.0,0.0,...,19.55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,9.61,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
%cd '/content/drive/MyDrive/machine_hack'

/content/drive/MyDrive/machine_hack


In [ ]:
movie_features_df.to_csv('sparce_app.csv',index=False)

In [ ]:
### Train-Test split
from scipy import sparse
x = sparse.csr_matrix(movie_features_df.values)  
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)

### Train the Matrix Factorization Model  
model = LightFM(no_components=50, loss='warp')
model.fit(x, epochs=30, num_threads = 4)
 
train_auc = auc_score(model, train, num_threads=4).mean()  
test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
print('Train AUC: %f, test AUC: %f'%(train_auc,test_auc))

Train AUC: 0.929659, test AUC: 0.927799


In [ ]:
preference = pd.DataFrame(model.predict(0, np.arange(movie_features_df.shape[1])))

In [ ]:
len(preference)

7005

In [ ]:
movie_features_df.index

Int64Index([    11,     15,     18,     22,     27,     48,     50,     56,
                71,     78,
            ...
            301241, 301316, 301317, 301318, 301330, 301336, 301343, 301344,
            301347, 301348],
           dtype='int64', name='uid', length=40920)

In [ ]:
preference.head()

,0,item_id
0,-2.532957,701
1,-2.629666,1048
2,-1.505238,5115
3,2.089459,5526
4,-3.772280,5536


In [ ]:
sort_values(by=['Country'])

In [ ]:
preference_new = preference.sort_values(by=[0],ascending=False)

In [ ]:
preference_new.head()

,0,item_id
28,4.015860,20875
1498,2.963165,145569
3549,2.960468,589378
3558,2.667255,589482
52,2.544383,25349


In [ ]:
aa[aa.uid==11]

In [ ]:
df_n[(df_n.uid==11)&(df_n.item_id==25349)]

In [ ]:
def user_item(x):
  

In [ ]:
movie_features_df.index[0]

11

In [ ]:
df_n.head()

,uid,item_id,sum,app_count,total_user_Count
0,11,5526,5.058157,113119,94
1,11,20875,3.020657,221507,94
2,11,28125,3.015725,7759,94
3,11,28297,3.000780,31088,94
4,11,28320,1.517739,4535,94


In [ ]:
df_n= df_n.reset_index(drop=True)


In [ ]:
aa_new=aa[aa.uid.isin(y) == False]

In [ ]:
gh=[]

In [ ]:
for i in range(len(movie_features_df.index)):
  preference = pd.DataFrame(model.predict(i, np.arange(movie_features_df.shape[1])))
  preference['item_id'] = movie_features_df.columns
  preference_new = preference.sort_values(by=[0],ascending=False)
  preference_new = preference_new.reset_index(drop=True)
  user = movie_features_df.index[i]
  need = df_n[df_n.uid==user]
  need = list(need['item_id'].values)
  #print(need)
  df_w=preference_new[preference_new['item_id'].isin(need) == False]
  #print(df_w)
  df_w = list(df_w['item_id'].values)
  df_w = df_w[0:4]
  gh.append(df_w)
  #print(df_w)
  #break


  


In [ ]:
gh_new=[]
for i in range(len(movie_features_df.index)):
  preference = pd.DataFrame(model.predict(i, np.arange(movie_features_df.shape[1])))
  preference['item_id'] = movie_features_df.columns
  preference_new = preference.sort_values(by=[0],ascending=False)
  preference_new = preference_new.reset_index(drop=True)
  #user = movie_features_df.index[i]
  #need = df_n[df_n.uid==user]
  #need = list(need['item_id'].values)
  #print(need)
  #df_w=preference_new[preference_new['item_id'].isin(need) == False]
  #print(df_w)
  df_w = list(preference_new['item_id'].values)
  df_w = df_w[0:4]
  gh_new.append(df_w)

In [ ]:
len(gh_new)

49593

In [ ]:
len(gh_new)

In [ ]:
gh

In [ ]:
len(movie_features_df.index)

40920

In [ ]:
for i in range (10):
  print(i)

In [ ]:
flatten_list = [j for sub in gh_new for j in sub]

In [ ]:
yui= df_n['uid'].unique()

In [ ]:
yui

array([    11,     15,     18, ..., 301344, 301347, 301348])

In [ ]:
nn= val[val['uid'].isin(yui) == False]

In [ ]:
nn

In [ ]:
app_u[app_u.uid==221103]

,uid,item_id,time_spent


In [ ]:
nn=nn.reset_index(drop=True)

In [ ]:
s = pd.Series([20875,25349,5526,145569])
bb=s.repeat()

In [ ]:
rt=[20875,25349,5526,145569]
mm = pd.Series(rt)

mm = mm.repeat(4)

In [ ]:
ss_uid = []
ss_item = []
for uid in ty:
  for item in rt:
      ss_uid.append(uid)
      ss_item.append(item)

In [ ]:
ss_uid

In [ ]:
ss_item

In [ ]:
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item

In [ ]:
nn_new.head()

,uid,item_id
0,14601,20875
1,14601,25349
2,14601,5526
3,14601,145569
4,221103,20875


In [ ]:
df_test=pd.concat([real,nn_new],axis=0)
df_test.head()

,uid,item_id
0,11,20875
1,11,822390
2,11,25349
3,11,145569
4,15,20875


In [ ]:
df_test.shape

(169008, 2)

In [ ]:
df_test.head()

,uid,item_id
0,11,20875
1,11,822390
2,11,25349
3,11,145569
4,15,20875


In [ ]:
ss.head()

,uid,item_id
0,255352,822587
1,290183,763460
2,210428,841722
3,17401,23802
4,48078,152446


In [ ]:
ss.shape

(169008, 2)

In [ ]:
df_test=df_test.reset_index(drop=True)
df_test.shape

(169008, 2)

In [ ]:
df_test.to_csv('full_avil.csv',index=False)

In [ ]:
ty = nn['uid']
tt = pd.Series(ty)

tt = tt.repeat(4)

In [ ]:
tt

0        14439
0        14439
0        14439
0        14439
1       195761
         ...  
1330    208776
1331    186841
1331    186841
1331    186841
1331    186841
Name: uid, Length: 5328, dtype: int64

In [ ]:
t = movie_features_df.index
ww = pd.Series(t)

ww = ww.repeat(4)

In [ ]:
len(ww)

198372

In [ ]:
len(flatten_list)

198372

In [ ]:
last['item_id'] = flatten_list

In [ ]:
last = pd.DataFrame()
last['uid'] = ww
last['item_id'] = flatten_list

In [ ]:
real = last[last.uid.isin(val['uid']) == True]

In [ ]:
real.head()

,uid,item_id
0,11,20875
0,11,822390
0,11,25349
0,11,145569
2,15,20875


In [ ]:
real=real.reset_index(drop=True)

In [ ]:
real.shape

(168800, 2)

In [ ]:
val.shape

(42252, 1)

In [ ]:
42252*4

169008

In [ ]:
169008-163680

5328

In [ ]:
len(flatten_list)

163680

In [ ]:
len(movie_features_df.index)

40920

In [ ]:
40920*4

163680

In [ ]:
preference['item_id'] = movie_features_df.columns

In [ ]:
preference.values

In [ ]:
len(movie_features_df.columns)

7005

In [ ]:
preference

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
  
def get_user_preference(x):  
    # Create a dictionary that stores the preference for each artist as a number between 0 and 1
    global preference, df_pref
    preference = pd.Series(model.predict(x, np.arange(input_matrix.shape[1]))).to_dict()
    lower_score = min(preference.values())
    highest_score = max(preference.values())
    preference = {artists_dict[i]:(preference[i]-lower_score)/(highest_score-lower_score) for i in preference}  
     
    # Print the known likes of the user
    known_items = list(pd.Series(input_matrix.loc[x,:][input_matrix.loc[x,:] > 0].index).sort_values(ascending=False))
    known_items = list(pd.Series(known_items).apply(lambda x: artists_dict[x])) 
    print("Top 20 artists this user already likes:\n",known_items[:20]) 
     
    # Print the predicted preference scores of the user
    df_pref = pd.DataFrame.from_records([(k, v) for k, v in preference.items()], columns =['artist', 'preference']) 
    df_pref = df_artist.merge(df_pref,left_on='artist',right_on='artist').sort_values(by = 'preference',ascending=False)
    print("\nPredicted preferences:") 

n_users = df_playlist['user_id'].nunique()
print("Select a user id from 0 through %i."%(n_users-1))
interact(get_user_preference, x=list(range(n_users)))
df_pref.head(30)